In [ ]:
%matplotlib inline

In [ ]:
from os import listdir
from os.path import isfile, join

import matplotlib.pyplot as plt
import numpy as np
from scipy import misc
import tensorflow as tf
from tqdm import tqdm_notebook

from preprocessing import _int64_feature, _bytes_feature, pad_image

In [ ]:
main_directory = "data"
data_directories = "clear  clear2  negativeSamples  residential  rotated10deg  rotated20deg  unclear  unclear2".split()
data_directories = [join(main_directory, d) for d in data_directories]

In [ ]:
for d in tqdm_notebook(data_directories, desc="Directories"):
    if d == "negativeSamples":
        label = 0
    elif d == "residential":
        label = 10
    else:
        label = 0
    for f in tqdm_notebook(listdir(d)):
        if isfile(join(d, f)) and f.split(".")[-1] == "jpg":
            image_path = join(d, f)
            image_name = ".".join(f.split(".")[:-1])
            image = misc.imread(image_path)
            tf_filename = join(d, image_name + ".tfrecords")
            writer = tf.python_io.TFRecordWriter(tf_filename)
            # we are going to pad here
            padded_image = pad_image(image)
            image_raw = padded_image.tostring()
            example = tf.train.Example(features=tf.train.Features(feature={
                "label": _int64_feature(label),
                "image_raw": _bytes_feature(image_raw)}))
            writer.write(example.SerializeToString())
            writer.close()